# CSV to Vdata

## Table of content

* 1-Import
* 2 - Ensembl id is unknown
* 3 - Proteins with at least two ensembl ids
* 4 - Several proteins for one gene
* 5 - Var dataframe
* 6- raw layer
* 7 - Mean layer
* 8 - replicates layers
* 9 - test VData reading

## 1- Import

In [1]:
import pandas as pd
import vdata
from vdata import TemporalDataFrame, VData, setLoggingLevel

Proteomic data from the step 1

In [3]:
proteomic_df = pd.read_csv(
    "/home/matteo/serveur_vidium/DMD/1-ExperimentalData/Proteomic_data/steps/01-Protein_to_ENSEMBL_ID/results"
    "/Proteomic_data_ENSEMBL_ID_curated.csv",
    sep=",",
    index_col="id",
)

In [4]:
proteomic_df.head()

,All_protein_ID,Protein names,Gene names,Mol. weight [kDa],Razor + unique peptides R1,Razor + unique peptides R2,Razor + unique peptides R3,Unique + razor sequence coverage [%],ANOVA Significant (1% BH-FDR)),ANOVA q-value,...,rawR3-WT9,rawR3-Mut9,rawR3-WT10,rawR3-Mut10,rawR3-WT12,rawR3-Mut12,rawR3-WT14,rawR3-Mut14,All_ENSEMBL_ID,Main_ensembl_id
id,,,,,,,,,,,,,,,,,,,,,
0,Q9H0G5;A0A024QZ33,Nuclear speckle splicing regulatory protein 1,NSRP1;CCDC55,"66,389",1,2,2,5,NaN,"9,25E-01",...,92848,112510,136290,128080,118650,134610,214620,133270,ENSG00000126653,ENSG00000126653
1,P35237;A0A2R8YD12;A0A024QZX5;A0A087X1N8;A0A2R8...,Serpin B6,SERPINB6,"42,621",2,0,0,"9,3",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000124570,ENSG00000124570
2,Q00341;A0A024R4E5;H0Y394,Vigilin,HDLBP,"141,45",27,23,28,"35,1",NaN,"1,00E+00",...,1764200,1653100,1988400,1844300,1854200,1677400,2280200,2400300,ENSG00000115677,ENSG00000115677
3,Q9H4M9;A0A024R571;C9JC03;C9J2Z4,EH domain-containing protein 1,EHD1,"60,626",17,16,13,"51,7",+,"1,01E-03",...,679770,588170,776850,637960,752200,645690,848520,978810,ENSG00000110047,ENSG00000110047
4,P25787;A0A024RA52;A0A7I2V2H3,Proteasome subunit alpha type-2;Proteasome sub...,PSMA2,"25,898",9,8,7,"35,9",NaN,"1,00E+00",...,459200,519630,484740,598120,485480,569490,638210,617400,ENSG00000106588;ENSG00000256646,ENSG00000106588


We only keep raw count data.

In [5]:
proteomic_df = proteomic_df[
    proteomic_df.columns[
        ((proteomic_df.columns.str[0:3] == "raw") == True)
        | ((proteomic_df.columns.str[:] == "Main_ensembl_id") == True)
        | ((proteomic_df.columns.str[:] == "Gene names") == True)
    ]
]
proteomic_df = proteomic_df.apply(lambda x: x.str.replace(",", "."))
proteomic_df[
    proteomic_df.columns[(proteomic_df.columns.str[0:3] == "raw") == True]
] = proteomic_df[
    proteomic_df.columns[(proteomic_df.columns.str[0:3] == "raw") == True].tolist()
].astype(
    float
)

proteomic_df.head()

,Gene names,rawR1-WT5,rawR1-Mut5,rawR1-WT6,rawR1-Mut6,rawR1-WT7,rawR1-Mut7,rawR1-WT8,rawR1-Mut8,rawR1-WT9,...,rawR3-Mut8,rawR3-WT9,rawR3-Mut9,rawR3-WT10,rawR3-Mut10,rawR3-WT12,rawR3-Mut12,rawR3-WT14,rawR3-Mut14,Main_ensembl_id
id,,,,,,,,,,,,,,,,,,,,,
0,NSRP1;CCDC55,112150.0,100250.0,102800.0,115760.0,93172.0,115860.0,105610.0,106860.0,94802.0,...,96349.0,92848.0,112510.0,136290.0,128080.0,118650.0,134610.0,214620.0,133270.0,ENSG00000126653
1,SERPINB6,40426.0,34249.0,44000.0,43406.0,33604.0,38418.0,42710.0,39058.0,43591.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000124570
2,HDLBP,1351100.0,1226800.0,1524000.0,1585600.0,1331200.0,1535400.0,1618700.0,1366300.0,1442700.0,...,1697200.0,1764200.0,1653100.0,1988400.0,1844300.0,1854200.0,1677400.0,2280200.0,2400300.0,ENSG00000115677
3,EHD1,554030.0,553380.0,627240.0,689730.0,628120.0,708150.0,795010.0,671280.0,783310.0,...,590670.0,679770.0,588170.0,776850.0,637960.0,752200.0,645690.0,848520.0,978810.0,ENSG00000110047
4,PSMA2,632690.0,614100.0,690080.0,727490.0,587610.0,731020.0,662280.0,687820.0,592640.0,...,540440.0,459200.0,519630.0,484740.0,598120.0,485480.0,569490.0,638210.0,617400.0,ENSG00000106588


In [6]:
proteomic_df = proteomic_df.rename(
    columns={
        "rawR1-Mut14": "rawR1-WT14",
        "rawR2-Mut14": "rawR2-WT14",
        "rawR3-Mut14": "rawR3-WT14",
        "rawR1-WT14": "rawR1-Mut14",
        "rawR2-WT14": "rawR2-Mut14",
        "rawR3-WT14": "rawR3-Mut14",
    }
)

There are few ambiguous cases :
- One row (so one protein ) could have several gene names and ensembl ids. In this cases we think that we can't trust the protein identification, so we remove rows.
- Several proteins can share one gene in common. As our analysis are focus on gene level, we chose to add counts of proteins sharing sames genes.
- The ensembl id is unknown


### 2 - Ensembl id is unknown

For these 3 genes the main ensembl id (based on hgnc is unknown), we removes rows

In [7]:
proteomic_df[proteomic_df.Main_ensembl_id.isna() == True]

,Gene names,rawR1-WT5,rawR1-Mut5,rawR1-WT6,rawR1-Mut6,rawR1-WT7,rawR1-Mut7,rawR1-WT8,rawR1-Mut8,rawR1-WT9,...,rawR3-Mut8,rawR3-WT9,rawR3-Mut9,rawR3-WT10,rawR3-Mut10,rawR3-WT12,rawR3-Mut12,rawR3-Mut14,rawR3-WT14,Main_ensembl_id
id,,,,,,,,,,,,,,,,,,,,,
422,HLA-C;HLA-Cw,25236.0,34094.0,22327.0,37721.0,19715.0,32602.0,28897.0,31499.0,23572.0,...,24989.0,20656.0,26159.0,24164.0,30933.0,24456.0,26834.0,42252.0,26472.0,NaN
423,HLA-A,96311.0,48499.0,99515.0,56060.0,85684.0,54221.0,97329.0,55649.0,89772.0,...,36288.0,66538.0,35510.0,78042.0,45924.0,72245.0,43213.0,61664.0,84730.0,NaN
5548,L1RE1,11675.0,8410.5,11608.0,12965.0,8512.3,11197.0,10573.0,11944.0,11624.0,...,10395.0,13198.0,11408.0,14920.0,12642.0,14824.0,14101.0,18759.0,17082.0,NaN


In [8]:
proteomic_df = proteomic_df[proteomic_df.Main_ensembl_id.isna() == False]
proteomic_df

,Gene names,rawR1-WT5,rawR1-Mut5,rawR1-WT6,rawR1-Mut6,rawR1-WT7,rawR1-Mut7,rawR1-WT8,rawR1-Mut8,rawR1-WT9,...,rawR3-Mut8,rawR3-WT9,rawR3-Mut9,rawR3-WT10,rawR3-Mut10,rawR3-WT12,rawR3-Mut12,rawR3-Mut14,rawR3-WT14,Main_ensembl_id
id,,,,,,,,,,,,,,,,,,,,,
0,NSRP1;CCDC55,112150.0,100250.0,102800.0,115760.0,93172.0,115860.0,105610.0,106860.0,94802.0,...,96349.0,92848.0,112510.0,136290.0,128080.0,118650.0,134610.0,214620.0,133270.0,ENSG00000126653
1,SERPINB6,40426.0,34249.0,44000.0,43406.0,33604.0,38418.0,42710.0,39058.0,43591.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000124570
2,HDLBP,1351100.0,1226800.0,1524000.0,1585600.0,1331200.0,1535400.0,1618700.0,1366300.0,1442700.0,...,1697200.0,1764200.0,1653100.0,1988400.0,1844300.0,1854200.0,1677400.0,2280200.0,2400300.0,ENSG00000115677
3,EHD1,554030.0,553380.0,627240.0,689730.0,628120.0,708150.0,795010.0,671280.0,783310.0,...,590670.0,679770.0,588170.0,776850.0,637960.0,752200.0,645690.0,848520.0,978810.0,ENSG00000110047
4,PSMA2,632690.0,614100.0,690080.0,727490.0,587610.0,731020.0,662280.0,687820.0,592640.0,...,540440.0,459200.0,519630.0,484740.0,598120.0,485480.0,569490.0,638210.0,617400.0,ENSG00000106588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5738,IVNS1ABP,95734.0,117470.0,105900.0,138760.0,85028.0,106300.0,105030.0,101820.0,76005.0,...,103620.0,79397.0,108470.0,91254.0,120470.0,81211.0,115940.0,129670.0,93933.0,ENSG00000116679
5739,SEC23IP,690970.0,678470.0,754330.0,835280.0,635770.0,770690.0,737600.0,692510.0,689790.0,...,637280.0,583910.0,642950.0,749620.0,735570.0,692760.0,730010.0,1019000.0,851350.0,ENSG00000107651
5740,COMMD3-BMI1;BMI1,74062.0,73028.0,77998.0,80546.0,69430.0,86929.0,85239.0,79020.0,81838.0,...,106750.0,102060.0,120740.0,154340.0,135800.0,136330.0,139650.0,209550.0,147950.0,ENSG00000269897;ENSG00000168283


## 3 - Proteins with at least two ensembl ids
We think we cannot distinguish proteins with the proteomic experiment and aren't confident. So we remove these rows.

In [9]:
proteomic_gene_df = proteomic_df[["Main_ensembl_id"]].astype(str)
proteomic_gene_df

,Main_ensembl_id
id,
0,ENSG00000126653
1,ENSG00000124570
2,ENSG00000115677
3,ENSG00000110047
4,ENSG00000106588
...,...
5738,ENSG00000116679
5739,ENSG00000107651
5740,ENSG00000269897;ENSG00000168283


We want to check if genes with multiple ensembl ids have multiple gene names

In [10]:
def check_multi_gene(row, list_multi_gene_prot, name_col):
    list_gene = row[name_col].split(";")
    if len(list_gene) > 1:
        list_multi_gene_prot.append(row.id)
    return

In [11]:
list_multi_ensembl = []
proteomic_gene_df.reset_index().apply(
    lambda row: check_multi_gene(row, list_multi_ensembl, "Main_ensembl_id"), axis=1
)
len(list_multi_ensembl)

218

There are 218 proteins with at least two ensembl id

In [12]:
prot_df = proteomic_df.reset_index()[["id", "Gene names"]]
prot_df = prot_df.dropna()
list_multi_gene_name = []
prot_df.apply(
    lambda row: check_multi_gene(row, list_multi_gene_name, "Gene names"), axis=1
)
len(list_multi_gene_name)

264

There are 264 protein with at least 2 gene names

List of proteins with multiple ensembl ids but only one gene name.

In [13]:
set(list_multi_ensembl) - set(list_multi_gene_name)

{193, 412, 722, 806, 1061, 2044, 2675, 2676, 2881, 3393, 4182, 5515, 5630}

We remove genes with several ensembl ids

In [14]:
proteomic_df = proteomic_df[proteomic_df.index.isin(list_multi_ensembl) == False]
proteomic_df

,Gene names,rawR1-WT5,rawR1-Mut5,rawR1-WT6,rawR1-Mut6,rawR1-WT7,rawR1-Mut7,rawR1-WT8,rawR1-Mut8,rawR1-WT9,...,rawR3-Mut8,rawR3-WT9,rawR3-Mut9,rawR3-WT10,rawR3-Mut10,rawR3-WT12,rawR3-Mut12,rawR3-Mut14,rawR3-WT14,Main_ensembl_id
id,,,,,,,,,,,,,,,,,,,,,
0,NSRP1;CCDC55,112150.0,100250.0,102800.0,115760.0,93172.0,115860.0,105610.0,106860.0,94802.0,...,96349.0,92848.0,112510.0,136290.0,128080.0,118650.0,134610.0,214620.0,133270.0,ENSG00000126653
1,SERPINB6,40426.0,34249.0,44000.0,43406.0,33604.0,38418.0,42710.0,39058.0,43591.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000124570
2,HDLBP,1351100.0,1226800.0,1524000.0,1585600.0,1331200.0,1535400.0,1618700.0,1366300.0,1442700.0,...,1697200.0,1764200.0,1653100.0,1988400.0,1844300.0,1854200.0,1677400.0,2280200.0,2400300.0,ENSG00000115677
3,EHD1,554030.0,553380.0,627240.0,689730.0,628120.0,708150.0,795010.0,671280.0,783310.0,...,590670.0,679770.0,588170.0,776850.0,637960.0,752200.0,645690.0,848520.0,978810.0,ENSG00000110047
4,PSMA2,632690.0,614100.0,690080.0,727490.0,587610.0,731020.0,662280.0,687820.0,592640.0,...,540440.0,459200.0,519630.0,484740.0,598120.0,485480.0,569490.0,638210.0,617400.0,ENSG00000106588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5736,FAM169A,86450.0,81010.0,79561.0,87633.0,72422.0,80007.0,86796.0,66521.0,67492.0,...,161910.0,158640.0,148710.0,185620.0,156070.0,146580.0,155410.0,206360.0,179540.0,ENSG00000198780
5738,IVNS1ABP,95734.0,117470.0,105900.0,138760.0,85028.0,106300.0,105030.0,101820.0,76005.0,...,103620.0,79397.0,108470.0,91254.0,120470.0,81211.0,115940.0,129670.0,93933.0,ENSG00000116679
5739,SEC23IP,690970.0,678470.0,754330.0,835280.0,635770.0,770690.0,737600.0,692510.0,689790.0,...,637280.0,583910.0,642950.0,749620.0,735570.0,692760.0,730010.0,1019000.0,851350.0,ENSG00000107651


Now we have 5414 proteins

### 4 - Several proteins for one gene

In [15]:
list_dupli_ens_id = (
    proteomic_df[proteomic_df.Main_ensembl_id.duplicated(keep=False) == True]
    .Main_ensembl_id.unique()
    .tolist()
)
len(list_dupli_ens_id)

52

There are 52 duplicated genes. One example below.

In [16]:
proteomic_df[proteomic_df.Main_ensembl_id == "ENSG00000143549"]

,Gene names,rawR1-WT5,rawR1-Mut5,rawR1-WT6,rawR1-Mut6,rawR1-WT7,rawR1-Mut7,rawR1-WT8,rawR1-Mut8,rawR1-WT9,...,rawR3-Mut8,rawR3-WT9,rawR3-Mut9,rawR3-WT10,rawR3-Mut10,rawR3-WT12,rawR3-Mut12,rawR3-Mut14,rawR3-WT14,Main_ensembl_id
id,,,,,,,,,,,,,,,,,,,,,
88,TPM3,1839100.0,1692700.0,1592800.0,1714300.0,1438900.0,1714100.0,1690100.0,1568600.0,1525500.0,...,1941400.0,1875200.0,1927900.0,2773400.0,2092200.0,2108800.0,2017400.0,3308900.0,2741800.0,ENSG00000143549
1944,TPM3,74029.0,71527.0,62834.0,64963.0,60511.0,77468.0,53482.0,60553.0,52209.0,...,54626.0,48172.0,61561.0,66844.0,74669.0,51672.0,67003.0,76524.0,65664.0,ENSG00000143549


We separate unique and duplicated genes.

In [17]:
proteomic_df_unique = proteomic_df[
    proteomic_df.Main_ensembl_id.isin(list_dupli_ens_id) == False
]
proteomic_df_unique

,Gene names,rawR1-WT5,rawR1-Mut5,rawR1-WT6,rawR1-Mut6,rawR1-WT7,rawR1-Mut7,rawR1-WT8,rawR1-Mut8,rawR1-WT9,...,rawR3-Mut8,rawR3-WT9,rawR3-Mut9,rawR3-WT10,rawR3-Mut10,rawR3-WT12,rawR3-Mut12,rawR3-Mut14,rawR3-WT14,Main_ensembl_id
id,,,,,,,,,,,,,,,,,,,,,
0,NSRP1;CCDC55,112150.0,100250.0,102800.0,115760.0,93172.0,115860.0,105610.0,106860.0,94802.0,...,96349.0,92848.0,112510.0,136290.0,128080.0,118650.0,134610.0,214620.0,133270.0,ENSG00000126653
1,SERPINB6,40426.0,34249.0,44000.0,43406.0,33604.0,38418.0,42710.0,39058.0,43591.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000124570
2,HDLBP,1351100.0,1226800.0,1524000.0,1585600.0,1331200.0,1535400.0,1618700.0,1366300.0,1442700.0,...,1697200.0,1764200.0,1653100.0,1988400.0,1844300.0,1854200.0,1677400.0,2280200.0,2400300.0,ENSG00000115677
3,EHD1,554030.0,553380.0,627240.0,689730.0,628120.0,708150.0,795010.0,671280.0,783310.0,...,590670.0,679770.0,588170.0,776850.0,637960.0,752200.0,645690.0,848520.0,978810.0,ENSG00000110047
4,PSMA2,632690.0,614100.0,690080.0,727490.0,587610.0,731020.0,662280.0,687820.0,592640.0,...,540440.0,459200.0,519630.0,484740.0,598120.0,485480.0,569490.0,638210.0,617400.0,ENSG00000106588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5735,WASF2,167980.0,170560.0,176340.0,191240.0,159770.0,191140.0,201580.0,177900.0,164940.0,...,285430.0,251390.0,271040.0,357900.0,317720.0,282600.0,296670.0,433370.0,341080.0,ENSG00000158195
5736,FAM169A,86450.0,81010.0,79561.0,87633.0,72422.0,80007.0,86796.0,66521.0,67492.0,...,161910.0,158640.0,148710.0,185620.0,156070.0,146580.0,155410.0,206360.0,179540.0,ENSG00000198780
5738,IVNS1ABP,95734.0,117470.0,105900.0,138760.0,85028.0,106300.0,105030.0,101820.0,76005.0,...,103620.0,79397.0,108470.0,91254.0,120470.0,81211.0,115940.0,129670.0,93933.0,ENSG00000116679


In [18]:
for ens_id in list_dupli_ens_id:  # for each gene in the list of duplicated gene
    dupli_id_df = proteomic_df[(proteomic_df.Main_ensembl_id == ens_id) == True]
    # We sum protein counts
    dupli_id_df_sum = pd.DataFrame(
        dupli_id_df[
            dupli_id_df.columns[((dupli_id_df.columns.str[0:3] == "raw") == True)]
        ].sum()
    ).transpose()

    dupli_id_df_concat = pd.concat(
        [
            dupli_id_df[["Gene names", "Main_ensembl_id"]][:1].reset_index(),
            dupli_id_df_sum,
        ],
        axis=1,
        join="outer",
    )
    id_multi = ""
    for k in dupli_id_df.index.tolist():
        id_multi = id_multi + str(k) + ";"
    id_multi[:-1]
    dupli_id_df_concat.loc[:, "id"] = id_multi[:-1]
    dupli_id_df_concat = dupli_id_df_concat.set_index("id")
    proteomic_df_unique = pd.concat([proteomic_df_unique, dupli_id_df_concat], axis=0)

In [19]:
dupli_id_df

,Gene names,rawR1-WT5,rawR1-Mut5,rawR1-WT6,rawR1-Mut6,rawR1-WT7,rawR1-Mut7,rawR1-WT8,rawR1-Mut8,rawR1-WT9,...,rawR3-Mut8,rawR3-WT9,rawR3-Mut9,rawR3-WT10,rawR3-Mut10,rawR3-WT12,rawR3-Mut12,rawR3-Mut14,rawR3-WT14,Main_ensembl_id
id,,,,,,,,,,,,,,,,,,,,,
3494,CAP1,2015300.0,1742100.0,2092900.0,2151900.0,1894300.0,2207200.0,2109600.0,1990700.0,1832500.0,...,2168700.0,2002000.0,2009400.0,2156700.0,2236900.0,2014800.0,2137300.0,2639800.0,2653200.0,ENSG00000131236
4000,CAP1,32545.0,35966.0,39713.0,42326.0,33062.0,36922.0,36225.0,39256.0,34726.0,...,29139.0,25864.0,29423.0,34419.0,31999.0,30346.0,30583.0,44862.0,41314.0,ENSG00000131236


In [20]:
proteomic_df_unique

,Gene names,rawR1-WT5,rawR1-Mut5,rawR1-WT6,rawR1-Mut6,rawR1-WT7,rawR1-Mut7,rawR1-WT8,rawR1-Mut8,rawR1-WT9,...,rawR3-Mut8,rawR3-WT9,rawR3-Mut9,rawR3-WT10,rawR3-Mut10,rawR3-WT12,rawR3-Mut12,rawR3-Mut14,rawR3-WT14,Main_ensembl_id
id,,,,,,,,,,,,,,,,,,,,,
0,NSRP1;CCDC55,112150.0,100250.0,102800.0,115760.0,93172.0,115860.0,105610.0,106860.0,94802.0,...,96349.0,92848.0,112510.0,136290.0,128080.0,118650.0,134610.0,214620.0,133270.0,ENSG00000126653
1,SERPINB6,40426.0,34249.0,44000.0,43406.0,33604.0,38418.0,42710.0,39058.0,43591.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000124570
2,HDLBP,1351100.0,1226800.0,1524000.0,1585600.0,1331200.0,1535400.0,1618700.0,1366300.0,1442700.0,...,1697200.0,1764200.0,1653100.0,1988400.0,1844300.0,1854200.0,1677400.0,2280200.0,2400300.0,ENSG00000115677
3,EHD1,554030.0,553380.0,627240.0,689730.0,628120.0,708150.0,795010.0,671280.0,783310.0,...,590670.0,679770.0,588170.0,776850.0,637960.0,752200.0,645690.0,848520.0,978810.0,ENSG00000110047
4,PSMA2,632690.0,614100.0,690080.0,727490.0,587610.0,731020.0,662280.0,687820.0,592640.0,...,540440.0,459200.0,519630.0,484740.0,598120.0,485480.0,569490.0,638210.0,617400.0,ENSG00000106588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2630;4028,TPM2,2149079.8,1599522.0,2067257.2,1593945.1,1968964.5,1954417.0,2193545.8,1818441.5,2032629.7,...,1825918.2,2048413.5,1877639.7,3476546.6,2044167.5,2347192.2,1963483.3,3642082.0,3474800.0,ENSG00000198467
3074;3075,TMPO,1366240.0,1240150.0,1310350.0,1359200.0,1119360.0,1321010.0,1269360.0,1297450.0,1244880.0,...,1463830.0,1382880.0,1584690.0,1887750.0,1843980.0,1750390.0,1886770.0,2885060.0,1889890.0,ENSG00000120802
3281;4053,PSMD4,331080.0,296615.0,322474.0,349703.0,280905.0,336718.0,324638.0,316498.0,296653.0,...,458710.0,435810.0,447960.0,532100.0,513750.0,469430.0,512010.0,745610.0,574590.0,ENSG00000159352


We have 5360 proteins (5038 + 52)

## 5 - Var dataframe

In [21]:
proteomic_var_df = (
    proteomic_df_unique[["Main_ensembl_id", "Gene names"]]
    .astype(str)
    .reset_index()
    .set_index("Main_ensembl_id")
    .rename(columns={"Gene names": "short_name"})
)
proteomic_var_df.loc[:, "organism"] = "Homo sapiens"
proteomic_var_df

,id,short_name,organism
Main_ensembl_id,,,
ENSG00000126653,0,NSRP1;CCDC55,Homo sapiens
ENSG00000124570,1,SERPINB6,Homo sapiens
ENSG00000115677,2,HDLBP,Homo sapiens
ENSG00000110047,3,EHD1,Homo sapiens
ENSG00000106588,4,PSMA2,Homo sapiens
...,...,...,...
ENSG00000198467,2630;4028,TPM2,Homo sapiens
ENSG00000120802,3074;3075,TMPO,Homo sapiens
ENSG00000159352,3281;4053,PSMD4,Homo sapiens


## 6 - raw layer

In [22]:
proteomic_raw_df = proteomic_df_unique.reset_index()
proteomic_raw_df = proteomic_raw_df.set_index("Main_ensembl_id")
proteomic_raw_df


,id,Gene names,rawR1-WT5,rawR1-Mut5,rawR1-WT6,rawR1-Mut6,rawR1-WT7,rawR1-Mut7,rawR1-WT8,rawR1-Mut8,...,rawR3-WT8,rawR3-Mut8,rawR3-WT9,rawR3-Mut9,rawR3-WT10,rawR3-Mut10,rawR3-WT12,rawR3-Mut12,rawR3-Mut14,rawR3-WT14
Main_ensembl_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000126653,0,NSRP1;CCDC55,112150.0,100250.0,102800.0,115760.0,93172.0,115860.0,105610.0,106860.0,...,99239.0,96349.0,92848.0,112510.0,136290.0,128080.0,118650.0,134610.0,214620.0,133270.0
ENSG00000124570,1,SERPINB6,40426.0,34249.0,44000.0,43406.0,33604.0,38418.0,42710.0,39058.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000115677,2,HDLBP,1351100.0,1226800.0,1524000.0,1585600.0,1331200.0,1535400.0,1618700.0,1366300.0,...,1984500.0,1697200.0,1764200.0,1653100.0,1988400.0,1844300.0,1854200.0,1677400.0,2280200.0,2400300.0
ENSG00000110047,3,EHD1,554030.0,553380.0,627240.0,689730.0,628120.0,708150.0,795010.0,671280.0,...,692700.0,590670.0,679770.0,588170.0,776850.0,637960.0,752200.0,645690.0,848520.0,978810.0
ENSG00000106588,4,PSMA2,632690.0,614100.0,690080.0,727490.0,587610.0,731020.0,662280.0,687820.0,...,521490.0,540440.0,459200.0,519630.0,484740.0,598120.0,485480.0,569490.0,638210.0,617400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000198467,2630;4028,TPM2,2149079.8,1599522.0,2067257.2,1593945.1,1968964.5,1954417.0,2193545.8,1818441.5,...,2259976.7,1825918.2,2048413.5,1877639.7,3476546.6,2044167.5,2347192.2,1963483.3,3642082.0,3474800.0
ENSG00000120802,3074;3075,TMPO,1366240.0,1240150.0,1310350.0,1359200.0,1119360.0,1321010.0,1269360.0,1297450.0,...,1465710.0,1463830.0,1382880.0,1584690.0,1887750.0,1843980.0,1750390.0,1886770.0,2885060.0,1889890.0
ENSG00000159352,3281;4053,PSMD4,331080.0,296615.0,322474.0,349703.0,280905.0,336718.0,324638.0,316498.0,...,486690.0,458710.0,435810.0,447960.0,532100.0,513750.0,469430.0,512010.0,745610.0,574590.0


## 7 - Mean layer

For WT cells

In [23]:
for cond in ["5", "6", "7", "8", "9", "10", "12", "14"]:
    proteomic_rep = proteomic_raw_df[
        proteomic_raw_df.columns[((proteomic_raw_df.columns.str[8:] == cond) == True)]
    ]

    # If just 1 value
    proteomic_rep_na = proteomic_rep[
        (proteomic_rep.isna().sum(axis=1) >= 2) == True
    ].copy()
    proteomic_rep_na.loc[:, "mean_WT" + cond] = "NaN"
    proteomic_rep_na = proteomic_rep_na[["mean_WT" + cond]]
    ## If 2 or 3 values
    proteomic_rep_val = proteomic_rep[
        (proteomic_rep.isna().sum(axis=1) <= 1) == True
    ].copy()
    proteomic_rep_val.loc[:, "mean_WT" + cond] = proteomic_rep_val.mean(axis=1)
    proteomic_rep_val = proteomic_rep_val[["mean_WT" + cond]]
    proteomic_rep_raw = pd.concat([proteomic_rep_val, proteomic_rep_na]).reset_index()
    proteomic_raw_df = proteomic_raw_df.merge(
        proteomic_rep_raw, left_on="Main_ensembl_id", right_on="Main_ensembl_id"
    ).set_index("Main_ensembl_id")

For Mutant cells

In [24]:
for cond in ["5", "6", "7", "8", "9", "10", "12", "14"]:
    proteomic_rep = proteomic_raw_df[
        proteomic_raw_df.columns[((proteomic_raw_df.columns.str[9:] == cond) == True)]
    ]

    # If just 1 value
    proteomic_rep_na = proteomic_rep[
        (proteomic_rep.isna().sum(axis=1) >= 2) == True
    ].copy()
    proteomic_rep_na.loc[:, "mean_Mu" + cond] = "NaN"
    proteomic_rep_na = proteomic_rep_na[["mean_Mu" + cond]]
    ## If 2 or 3 values
    proteomic_rep_val = proteomic_rep[
        (proteomic_rep.isna().sum(axis=1) <= 1) == True
    ].copy()
    proteomic_rep_val.loc[:, "mean_Mu" + cond] = proteomic_rep_val.mean(axis=1)
    proteomic_rep_val = proteomic_rep_val[["mean_Mu" + cond]]
    proteomic_rep_raw = pd.concat([proteomic_rep_val, proteomic_rep_na]).reset_index()
    proteomic_raw_df = proteomic_raw_df.merge(
        proteomic_rep_raw, left_on="Main_ensembl_id", right_on="Main_ensembl_id"
    ).set_index("Main_ensembl_id")

In [25]:
proteomic_raw_df

,id,Gene names,rawR1-WT5,rawR1-Mut5,rawR1-WT6,rawR1-Mut6,rawR1-WT7,rawR1-Mut7,rawR1-WT8,rawR1-Mut8,...,mean_WT12,mean_WT14,mean_Mu5,mean_Mu6,mean_Mu7,mean_Mu8,mean_Mu9,mean_Mu10,mean_Mu12,mean_Mu14
Main_ensembl_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000126653,0,NSRP1;CCDC55,112150.0,100250.0,102800.0,115760.0,93172.0,115860.0,105610.0,106860.0,...,125880.0,143576.666667,99497.666667,121156.666667,115850.0,107396.333333,120460.0,137650.0,140793.333333,238420.0
ENSG00000124570,1,SERPINB6,40426.0,34249.0,44000.0,43406.0,33604.0,38418.0,42710.0,39058.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000115677,2,HDLBP,1351100.0,1226800.0,1524000.0,1585600.0,1331200.0,1535400.0,1618700.0,1366300.0,...,1528600.0,1929033.333333,1216723.333333,1568000.0,1511333.333333,1364966.666667,1329723.333333,1484700.0,1361033.333333,1850533.333333
ENSG00000110047,3,EHD1,554030.0,553380.0,627240.0,689730.0,628120.0,708150.0,795010.0,671280.0,...,811043.333333,1040583.333333,489693.333333,629270.0,658883.333333,610393.333333,608383.333333,673286.666667,693680.0,927080.0
ENSG00000106588,4,PSMA2,632690.0,614100.0,690080.0,727490.0,587610.0,731020.0,662280.0,687820.0,...,522066.666667,663040.0,521756.666667,620983.333333,626043.333333,585156.666667,563710.0,657853.333333,622463.333333,696580.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000198467,2630;4028,TPM2,2149079.8,1599522.0,2067257.2,1593945.1,1968964.5,1954417.0,2193545.8,1818441.5,...,2028406.4,2988677.933333,1410389.9,1424793.3,1739281.033333,1614441.966667,1642366.833333,1803812.866667,1724218.933333,3046555.533333
ENSG00000120802,3074;3075,TMPO,1366240.0,1240150.0,1310350.0,1359200.0,1119360.0,1321010.0,1269360.0,1297450.0,...,1671230.0,1784236.666667,1340403.333333,1458496.666667,1404260.0,1374850.0,1520496.666667,1764993.333333,1800810.0,2752200.0
ENSG00000159352,3281;4053,PSMD4,331080.0,296615.0,322474.0,349703.0,280905.0,336718.0,324638.0,316498.0,...,378179.333333,461521.333333,340171.666667,396224.333333,380219.333333,365522.666667,366679.0,422636.333333,412729.666667,592955.333333


In [26]:
raw_Mean_df = proteomic_raw_df[
    proteomic_raw_df.columns[(proteomic_raw_df.columns.str[:4] == "mean") == True]
]
raw_Mean_df = raw_Mean_df.transpose()
raw_Mean_df.index = [
    cond[5:7] for cond in list(raw_Mean_df.index)
]
raw_Mean_df.index = raw_Mean_df.index.rename("Cell_type")
raw_Mean_df


Main_ensembl_id,ENSG00000126653,ENSG00000124570,ENSG00000115677,ENSG00000110047,ENSG00000106588,ENSG00000011007,ENSG00000184047,ENSG00000204463,ENSG00000101974,ENSG00000144560,...,ENSG00000167004,ENSG00000067955,ENSG00000134440,ENSG00000127527,ENSG00000104833,ENSG00000198467,ENSG00000120802,ENSG00000159352,ENSG00000165119,ENSG00000131236
Cell_type,,,,,,,,,,,,,,,,,,,,,
WT,115223.333333,NaN,1342333.333333,519633.333333,529030.0,56806.666667,128469.0,653866.666667,44157.5,20133.0,...,8698420.0,385000.333333,1337760.0,351078.666667,22603.333333,1920295.266667,1468016.666667,387093.333333,5017060.0,2078241.333333
WT,115546.666667,NaN,1521900.0,578723.333333,586206.666667,57049.333333,147017.333333,704560.0,55676.5,24945.0,...,8398763.333333,420642.666667,1447163.333333,390998.333333,22852.666667,1857119.566667,1404506.666667,361831.333333,4807766.666667,2174395.0
WT,98533.333333,NaN,1323300.0,586043.333333,505453.333333,51996.333333,119352.666667,572913.333333,45193.0,21150.0,...,6901480.0,389426.666667,1183470.0,332227.333333,16308.333333,1751725.566667,1218740.0,327035.0,4246986.666667,1957941.666667
WT,113793.0,NaN,1606833.333333,725820.0,564583.333333,59662.0,156599.333333,666943.333333,58592.0,23207.0,...,8426196.666667,449403.0,1358206.666667,391372.0,20739.333333,1970926.766667,1368843.333333,383796.0,4750576.666667,2188630.333333
WT,104280.0,NaN,1435833.333333,717343.333333,502106.666667,53197.666667,147283.666667,615010.0,50895.0,22463.0,...,7921960.0,382891.666667,1137165.666667,366243.666667,19891.0,1815521.7,1323923.333333,345177.666667,4259006.666667,1903559.666667
WT,148806.666667,NaN,1619400.0,820133.333333,526530.0,73354.666667,207490.0,761780.0,62390.5,26798.0,...,11865903.333333,501809.333333,1197578.333333,474435.0,23543.666667,2969957.566667,1793303.333333,427224.666667,5657190.0,2079389.666667
WT,125880.0,NaN,1528600.0,811043.333333,522066.666667,63800.333333,166141.666667,745690.0,62122.0,26014.5,...,10913756.666667,431448.666667,1045498.0,429095.666667,23479.0,2028406.4,1671230.0,378179.333333,5211076.666667,1924111.333333
WT,143576.666667,NaN,1929033.333333,1040583.333333,663040.0,71504.666667,211233.333333,866370.0,76912.0,29110.5,...,12501663.333333,511669.333333,1344120.0,525599.333333,26544.666667,2988677.933333,1784236.666667,461521.333333,6037593.333333,2528436.333333
Mu,99497.666667,NaN,1216723.333333,489693.333333,521756.666667,53170.333333,127722.666667,633516.666667,43149.5,21710.0,...,7822283.333333,335707.666667,1236230.0,355379.666667,25782.333333,1410389.9,1340403.333333,340171.666667,4854566.666667,1840094.333333


In [27]:
raw_Mean_tmpdf = vdata.TemporalDataFrame(
    raw_Mean_df,
    time_list=[
        "5D",
        "6D",
        "7D",
        "8D",
        "9D",
        "10D",
        "12D",
        "14D",
        "5D",
        "6D",
        "7D",
        "8D",
        "9D",
        "10D",
        "12D",
        "14D",
    ],
)

## 8 - replicates layers

In [28]:
raw_R1_df = proteomic_raw_df[
    proteomic_raw_df.columns[(proteomic_raw_df.columns.str[3:5] == "R1") == True]
]
raw_R1_df = raw_R1_df.transpose()
raw_R1_df.index = [cond[6:8] for cond in list(raw_R1_df.index)]
raw_R1_df.index = raw_R1_df.index.rename("Cell_type")
raw_R1_df = raw_R1_df.sort_values("Cell_type", ascending=False)
raw_R1_df

Main_ensembl_id,ENSG00000126653,ENSG00000124570,ENSG00000115677,ENSG00000110047,ENSG00000106588,ENSG00000011007,ENSG00000184047,ENSG00000204463,ENSG00000101974,ENSG00000144560,...,ENSG00000167004,ENSG00000067955,ENSG00000134440,ENSG00000127527,ENSG00000104833,ENSG00000198467,ENSG00000120802,ENSG00000159352,ENSG00000165119,ENSG00000131236
Cell_type,,,,,,,,,,,,,,,,,,,,,
WT,112150.0,40426.0,1351100.0,554030.0,632690.0,61753.0,201520.0,628140.0,NaN,NaN,...,8534090.0,388448.0,1151800.0,332500.0,0.0,2149079.8,1366240.0,331080.0,4932500.0,2047845.0
WT,102800.0,44000.0,1524000.0,627240.0,690080.0,64703.0,227240.0,676760.0,NaN,NaN,...,8223460.0,423737.0,1234700.0,366590.0,0.0,2067257.2,1310350.0,322474.0,4736420.0,2132613.0
WT,93172.0,33604.0,1331200.0,628120.0,587610.0,60229.0,185190.0,559950.0,NaN,NaN,...,6799650.0,374218.0,1018900.0,314600.0,0.0,1968964.5,1119360.0,280905.0,4114530.0,1927362.0
WT,105610.0,42710.0,1618700.0,795010.0,662280.0,65747.0,252360.0,659730.0,NaN,NaN,...,8291150.0,446421.0,1148300.0,357990.0,0.0,2193545.8,1269360.0,324638.0,4656730.0,2145825.0
WT,94802.0,43591.0,1442700.0,783310.0,592640.0,60929.0,236160.0,605890.0,NaN,NaN,...,7784440.0,377559.0,966000.0,337830.0,0.0,2032629.7,1244880.0,296653.0,4110580.0,1867226.0
WT,135300.0,40201.0,1634700.0,915460.0,624130.0,77827.0,330750.0,747630.0,NaN,NaN,...,11577840.0,530991.0,1001400.0,433220.0,0.0,3284511.1,1641710.0,375424.0,5536030.0,1991905.0
WT,116930.0,46099.0,1562400.0,899380.0,622400.0,74710.0,268780.0,738600.0,NaN,NaN,...,10716030.0,446168.0,873970.0,398950.0,0.0,2259202.8,1507190.0,324438.0,5094970.0,1893358.0
WT,136980.0,52507.0,1934600.0,1154100.0,775110.0,80879.0,344300.0,849250.0,NaN,NaN,...,12261490.0,522002.0,1122400.0,483470.0,0.0,3327634.7,1627600.0,394714.0,5912320.0,2466958.0
Mu,100250.0,34249.0,1226800.0,553380.0,614100.0,54773.0,198140.0,608370.0,NaN,NaN,...,7649260.0,335064.0,1033400.0,332020.0,0.0,1599522.0,1240150.0,296615.0,4724920.0,1778066.0


In [29]:
raw_R2_df = proteomic_raw_df[
    proteomic_raw_df.columns[(proteomic_raw_df.columns.str[3:5] == "R2") == True]
]
raw_R2_df = raw_R2_df.transpose()
raw_R2_df.index = [cond[6:8] for cond in list(raw_R2_df.index)]
raw_R2_df.index = raw_R2_df.index.rename("Cell_type")

raw_R2_df = raw_R2_df.sort_values("Cell_type", ascending=False)
raw_R2_df

Main_ensembl_id,ENSG00000126653,ENSG00000124570,ENSG00000115677,ENSG00000110047,ENSG00000106588,ENSG00000011007,ENSG00000184047,ENSG00000204463,ENSG00000101974,ENSG00000144560,...,ENSG00000167004,ENSG00000067955,ENSG00000134440,ENSG00000127527,ENSG00000104833,ENSG00000198467,ENSG00000120802,ENSG00000159352,ENSG00000165119,ENSG00000131236
Cell_type,,,,,,,,,,,,,,,,,,,,,
WT,124970.0,NaN,1008700.0,495220.0,480130.0,52919.0,127020.0,675210.0,30752.0,19452.0,...,8472370.0,415033.0,1492370.0,284130.0,67810.0,1424178.4,1488200.0,340070.0,4673550.0,1984976.0
WT,132170.0,NaN,1162800.0,559490.0,523990.0,50537.0,147190.0,710850.0,37739.0,23314.0,...,8169340.0,446211.0,1594040.0,321560.0,68558.0,1417843.3,1421140.0,326300.0,4474710.0,2053793.0
WT,116930.0,NaN,1006200.0,560330.0,468470.0,45839.0,115000.0,582130.0,31891.0,19991.0,...,6667760.0,428512.0,1293790.0,275450.0,48925.0,1324548.8,1246070.0,297330.0,3969540.0,1851418.0
WT,136530.0,NaN,1217300.0,689750.0,509980.0,52662.0,136430.0,677710.0,41381.0,21338.0,...,8135980.0,478528.0,1496080.0,316010.0,62218.0,1459257.8,1371460.0,340060.0,4450570.0,2081384.0
WT,125190.0,NaN,1100600.0,688950.0,454480.0,46164.0,128740.0,614070.0,35377.0,20440.0,...,7635020.0,414136.0,1215019.0,305960.0,59673.0,1365521.9,1344010.0,303070.0,4006030.0,1815589.0
WT,174830.0,NaN,1235100.0,768090.0,470720.0,66253.0,161640.0,759610.0,42330.0,24498.0,...,11543550.0,516247.0,1290095.0,385960.0,70631.0,2148815.0,1850450.0,374150.0,5234240.0,2055145.0
WT,142060.0,NaN,1169200.0,781550.0,458320.0,54614.0,138200.0,755470.0,42876.0,24337.0,...,10522180.0,446328.0,1113817.0,346880.0,70437.0,1478824.2,1756110.0,340670.0,4783620.0,1833830.0
WT,160480.0,NaN,1452200.0,988840.0,596610.0,62045.0,171530.0,870300.0,53034.0,27154.0,...,12005670.0,536036.0,1460630.0,434310.0,79634.0,2163599.1,1835220.0,415260.0,5553360.0,2423837.0
Mu,105900.0,NaN,933070.0,455410.0,472690.0,49121.0,124920.0,647350.0,30307.0,21053.0,...,7609530.0,347149.0,1405400.0,292980.0,77347.0,1037475.1,1359560.0,303680.0,4605170.0,1772736.0


In [30]:
raw_R3_df = proteomic_raw_df[
    proteomic_raw_df.columns[(proteomic_raw_df.columns.str[3:5] == "R3") == True]
]
raw_R3_df = raw_R3_df.transpose()
raw_R3_df.index = [cond[6:8] for cond in list(raw_R3_df.index)]
raw_R3_df.index = raw_R3_df.index.rename("Cell_type")

raw_R3_df = raw_R3_df.sort_values("Cell_type", ascending=False)
raw_R3_df

Main_ensembl_id,ENSG00000126653,ENSG00000124570,ENSG00000115677,ENSG00000110047,ENSG00000106588,ENSG00000011007,ENSG00000184047,ENSG00000204463,ENSG00000101974,ENSG00000144560,...,ENSG00000167004,ENSG00000067955,ENSG00000134440,ENSG00000127527,ENSG00000104833,ENSG00000198467,ENSG00000120802,ENSG00000159352,ENSG00000165119,ENSG00000131236
Cell_type,,,,,,,,,,,,,,,,,,,,,
WT,108550.0,NaN,1667200.0,509650.0,474270.0,55748.0,56867.0,658250.0,57563.0,20814.0,...,9088800.0,351520.0,1369110.0,436606.0,0.0,2187627.6,1549610.0,490130.0,5445130.0,2201903.0
WT,111670.0,NaN,1878900.0,549440.0,544550.0,55908.0,66622.0,726070.0,73614.0,26576.0,...,8803490.0,391980.0,1512750.0,484845.0,0.0,2086258.2,1482030.0,436720.0,5212170.0,2336779.0
WT,85498.0,NaN,1632500.0,569680.0,460280.0,49921.0,57868.0,576660.0,58495.0,22309.0,...,7237030.0,365550.0,1237720.0,406632.0,0.0,1961663.4,1290790.0,402870.0,4656890.0,2095045.0
WT,99239.0,NaN,1984500.0,692700.0,521490.0,60577.0,81008.0,663390.0,75803.0,25076.0,...,8851460.0,423260.0,1430240.0,500116.0,0.0,2259976.7,1465710.0,486690.0,5144430.0,2338682.0
WT,92848.0,NaN,1764200.0,679770.0,459200.0,52500.0,76951.0,625070.0,66413.0,24486.0,...,8346420.0,356980.0,1230478.0,454941.0,0.0,2048413.5,1382880.0,435810.0,4660410.0,2027864.0
WT,136290.0,NaN,1988400.0,776850.0,484740.0,75984.0,130080.0,778100.0,82451.0,29098.0,...,12476320.0,458190.0,1301240.0,604125.0,0.0,3476546.6,1887750.0,532100.0,6201300.0,2191119.0
WT,118650.0,NaN,1854200.0,752200.0,485480.0,62077.0,91445.0,743000.0,81368.0,27692.0,...,11503060.0,401850.0,1148707.0,541457.0,0.0,2347192.2,1750390.0,469430.0,5754640.0,2045146.0
WT,133270.0,NaN,2400300.0,978810.0,617400.0,71590.0,117870.0,879560.0,100790.0,31067.0,...,13237830.0,476970.0,1449330.0,659018.0,0.0,3474800.0,1889890.0,574590.0,6647100.0,2694514.0
Mu,92343.0,NaN,1490300.0,460290.0,478480.0,55617.0,60108.0,644830.0,55992.0,22367.0,...,8208060.0,324910.0,1269890.0,441139.0,0.0,1594172.6,1421500.0,420220.0,5233610.0,1969481.0


In [31]:
raw_R1_tmpdf = vdata.TemporalDataFrame(
    raw_R1_df,
    time_list=[
        "5D",
        "6D",
        "7D",
        "8D",
        "9D",
        "10D",
        "12D",
        "14D",
        "5D",
        "6D",
        "7D",
        "8D",
        "9D",
        "10D",
        "12D",
        "14D",
    ],
)
raw_R2_tmpdf = vdata.TemporalDataFrame(
    raw_R1_df,
    time_list=[
        "5D",
        "6D",
        "7D",
        "8D",
        "9D",
        "10D",
        "12D",
        "14D",
        "5D",
        "6D",
        "7D",
        "8D",
        "9D",
        "10D",
        "12D",
        "14D",
    ],
)
raw_R3_tmpdf = vdata.TemporalDataFrame(
    raw_R1_df,
    time_list=[
        "5D",
        "6D",
        "7D",
        "8D",
        "9D",
        "10D",
        "12D",
        "14D",
        "5D",
        "6D",
        "7D",
        "8D",
        "9D",
        "10D",
        "12D",
        "14D",
    ],
)

## Vdata

In [32]:
raw_proteomic = {
    "R1": raw_R1_tmpdf,
    "R2": raw_R2_tmpdf,
    "R3": raw_R3_tmpdf,
    "Mean": raw_Mean_tmpdf,
}

In [33]:
obs = TemporalDataFrame(
    {"Cell_type": raw_R1_tmpdf.index},
    time_list=[
        "5D",
        "5D",
        "6D",
        "6D",
        "7D",
        "7D",
        "8D",
        "8D",
        "9D",
        "9D",
        "10D",
        "10D",
        "12D",
        "12D",
        "14D",
        "14D",
    ],
    index=[
        "WT",
        "Mu",
        "WT",
        "Mu",
        "WT",
        "Mu",
        "WT",
        "Mu",
        "WT",
        "Mu",
        "WT",
        "Mu",
        "WT",
        "Mu",
        "WT",
        "Mu",
    ],
    name="obs",
)

In [34]:
proteomic_raw_vdata = VData(raw_proteomic, obs=obs, var=proteomic_var_df)

In [35]:
proteomic_raw_vdata.var

,id,short_name,organism
Main_ensembl_id,,,
ENSG00000126653,0,NSRP1;CCDC55,Homo sapiens
ENSG00000124570,1,SERPINB6,Homo sapiens
ENSG00000115677,2,HDLBP,Homo sapiens
ENSG00000110047,3,EHD1,Homo sapiens
ENSG00000106588,4,PSMA2,Homo sapiens
...,...,...,...
ENSG00000198467,2630;4028,TPM2,Homo sapiens
ENSG00000120802,3074;3075,TMPO,Homo sapiens
ENSG00000159352,3281;4053,PSMD4,Homo sapiens


In [36]:
proteomic_raw_vdata.obs["14D"]

View of TemporalDataFrame 'obs'
Time point : 14.0 days
   Cell_type
WT        WT
Mu        Mu

[2 x 1]


In [37]:
type(proteomic_raw_vdata.layers["R1"]["5D", "WT", "ENSG00000101974"].values[0, 0])

numpy.float64

In [44]:
import numpy as np
proteomic_raw_vdata.layers["Mean"].astype(np.float64)

In [45]:
setLoggingLevel('INFO')
proteomic_raw_vdata.write("./../results/Proteomic_VData.h5")

16:25:51 :       INFO : [write.py] Saving dict layers
16:25:51 :       INFO : [write.py]   ↳ Saving TemporalDataFrame R1
16:25:51 :       INFO : [write.py]     ↳ Saving Series index
16:25:51 :       INFO : [write.py]     ↳ Saving None value for time_col_name
16:25:51 :       INFO : [write.py]     ↳ Saving Series columns
16:25:51 :       INFO : [write.py]     ↳ Saving time point 5.0D
16:25:52 :       INFO : [write.py]     ↳ Saving time point 6.0D
16:25:52 :       INFO : [write.py]     ↳ Saving time point 7.0D
16:25:52 :       INFO : [write.py]     ↳ Saving time point 8.0D
16:25:53 :       INFO : [write.py]     ↳ Saving time point 9.0D
16:25:53 :       INFO : [write.py]     ↳ Saving time point 10.0D
16:25:54 :       INFO : [write.py]     ↳ Saving time point 12.0D
16:25:54 :       INFO : [write.py]     ↳ Saving time point 14.0D
16:25:54 :       INFO : [write.py]   ↳ Saving TemporalDataFrame R2
16:25:54 :       INFO : [write.py]     ↳ Saving Series index
16:25:54 :       INFO : [write.py] 

## 9 - test VData reading

In [ ]:
vdata_prot_read = vdata.read_from_csv("./../results/Proteomic_VData")

In [ ]:
vdata_prot_read.var

In [ ]:
vdata_prot_read.layers["Mean"]